In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import os
import pandas as pd
import glob
pd.set_option('display.max_columns',40)
pd.set_option('display.max_rows', 300)  # Show all rows

os.chdir('/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/')
# Path to your data folder in Google Drive
data_folder = '/content/drive/MyDrive/Real_Trader_EDA_R1/Data/Target_Optimization_data'

# List files in the specified folder to confirm access
folder_path = os.path.join(data_folder, '/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data')  # Update 'path_to_your_data_folder' with your actual folder path
file_list = os.listdir(folder_path)
file_list

['Copy of ideal_target.csv',
 'Copy of New_5min_data (2).csv',
 'Copy of ES 09-24_Minute_5.csv',
 'df_target_comparision.csv',
 '.ipynb_checkpoints',
 'ideal_data_profit_loss_data.csv',
 'cal_data_profit_loss_data.csv',
 'trade_data_profit_loss_data.csv',
 'trade_data_processed.csv',
 'trade_defence_target.csv']

In [28]:
trade_data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/trade_data_profit_loss_data.csv")  #trade data
cal_data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/cal_data_profit_loss_data.csv")  #cal data
ideal_data = pd.read_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/ideal_data_profit_loss_data.csv")  #ideal

In [36]:
trade_data.tail(2)

,Time Stamp,Open,High,Low,Close,SellersLevel4,SellersLevel3,SellersLevel2,SellersLevel1,SellersLevel0,BuyersLevel0,BuyersLevel1,BuyersLevel2,BuyersLevel3,BuyersLevel4,BuyersInControlTime,BuyersInControlPrice,SellersInControlTime,SellersInControlPrice,Entry,Defence,Stop,Target,Profit,Loss,match_time_stamp,defence_target,Entry_Type
8309,2019-07-24 23:55:00,3304.75,3305.25,3304.75,3305.25,3305.25,3305.25,3305.25,3305.0,3305.0,3304.0,3304.0,3304.0,3304.0,3304.25,07/24/2019 22:45,3303.5,0,0.0,3303.5,3302.5,3301.5,3305.5,0.0,0.0,0,0.0,Long_Entry
8310,2019-07-24 23:55:00,3304.75,3305.25,3304.75,3305.25,3305.25,3305.25,3305.25,3305.0,3305.0,3304.0,3304.0,3304.0,3304.0,3304.25,07/24/2019 22:45,3303.5,0,0.0,3303.5,3302.5,3301.5,3305.5,0.0,0.0,0,0.0,Long_Entry


In [29]:
# Function to process a single CSV file
# Function to process a single CSV file
def process_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Check for unnamed columns and remove them if present
    unnamed_cols = df.columns.str.contains('^Unnamed')
    if unnamed_cols.any():
        df = df.loc[:, ~unnamed_cols]

    # Convert 'Time Stamp' column to datetime if it exists
    if 'Time Stamp' in df.columns:
        df['Time Stamp'] = pd.to_datetime(df['Time Stamp'])

    return df

# Paths to your CSV files
csv_files = ['trade_data_profit_loss_data.csv', 'cal_data_profit_loss_data.csv', 'ideal_data_profit_loss_data.csv']

# Process each CSV file
processed_dataframes = [process_csv(file) for file in csv_files]

cal_data = processed_dataframes[0]
trade_data = processed_dataframes[1]
ideal_data = processed_dataframes[2]


In [30]:
# Dropping columns from trade_data
trade_data.drop(columns=["Highest_value", "Highest_timestamp", "Stop_Match_timestamp"], inplace=True)

# Dropping columns from cal_data
cal_data.drop(columns=["Highest_value", "Highest_timestamp", "Stop_Match_timestamp"], inplace=True)

# Dropping columns from ideal_data
ideal_data.drop(columns=["Highest_value", "Highest_timestamp", "Stop_Match_timestamp"], inplace=True)


In [31]:
trade_data.columns

Index(['Time Stamp', 'Open', 'High', 'Low', 'Close', 'SellersLevel4',
       'SellersLevel3', 'SellersLevel2', 'SellersLevel1', 'SellersLevel0',
       'BuyersLevel0', 'BuyersLevel1', 'BuyersLevel2', 'BuyersLevel3',
       'BuyersLevel4', 'BuyersInControlTime', 'BuyersInControlPrice',
       'SellersInControlTime', 'SellersInControlPrice', 'Entry', 'Defence',
       'Stop', 'Target', 'Profit', 'Loss'],
      dtype='object')

In [32]:
import pandas as pd

def process_trade_data_mde(trade_data):
    # Initialize new columns with default values
    trade_data['match_time_stamp'] = 0
    trade_data['defence_target'] = 0
    trade_data['Entry_Type'] = 'No Entry'

    for i, row in trade_data.iterrows():
        entry = row['Entry']
        defence = row['Defence']
        buyers_in_control = row['BuyersInControlPrice']
        sellers_in_control = row['SellersInControlPrice']

        # Check for Long entry
        if not pd.isna(buyers_in_control) and buyers_in_control != 0:
            trade_data.at[i, 'Entry_Type'] = 'Long_Entry'
            # Look at future rows only for matching
            future_rows = trade_data.iloc[i+1:]
            match_index = future_rows[future_rows['Low'] == defence].index
            if not match_index.empty:
                match_index = match_index[0]
                trade_data.at[i, 'match_time_stamp'] = trade_data.at[match_index, 'Time Stamp']
                trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) + 1

        # Check for Short entry
        elif not pd.isna(sellers_in_control) and sellers_in_control != 0:
            trade_data.at[i, 'Entry_Type'] = 'Short_Entry'
            # Look at future rows only for matching
            future_rows = trade_data.iloc[i+1:]
            match_index = future_rows[future_rows['High'] == defence].index
            if not match_index.empty:
                match_index = match_index[0]
                trade_data.at[i, 'match_time_stamp'] = trade_data.at[match_index, 'Time Stamp']
                trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) - 1

    return trade_data

In [33]:
# Apply the function to the DataFrame
trade_data_processed = process_trade_data_mde(trade_data)
cal_data_processed = process_trade_data_mde(trade_data)
ideal_data_processed = process_trade_data_mde(trade_data)
trade_data_processed.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Reference_data/trade_defence_target_mde.csv", index=False)
ideal_data_processed.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Reference_data/ideal_defence_target_mde.csv", index=False)
cal_data_processed.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Reference_data/cal_defence_target_mde.csv", index=False)

<ipython-input-32-87d827d7190a>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-06-24 10:45:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_data.at[i, 'match_time_stamp'] = trade_data.at[match_index, 'Time Stamp']
<ipython-input-32-87d827d7190a>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3239.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) - 1
<ipython-input-32-87d827d7190a>:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2019-06-24 10:45:00' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_data.at[i, 'match_time_stamp'] = trade_data.at[match_index, 'Time Sta

In [34]:
import pandas as pd

def process_trade_data_defence(trade_data):
    # Initialize the 'defence_target' column with default value 0
    trade_data['defence_target'] = 0

    for i, row in trade_data.iterrows():
        entry = row['Entry']
        defence = row['Defence']
        buyers_in_control = row['BuyersInControlPrice']
        sellers_in_control = row['SellersInControlPrice']

        # Check for Long entry
        if not pd.isna(buyers_in_control) and buyers_in_control != 0:
            # Look at future rows only for matching
            future_rows = trade_data.iloc[i+1:]
            match_index = future_rows[future_rows['Low'] == defence].index
            if not match_index.empty:
                match_index = match_index[0]
                trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) + 1

        # Check for Short entry
        elif not pd.isna(sellers_in_control) and sellers_in_control != 0:
            # Look at future rows only for matching
            future_rows = trade_data.iloc[i+1:]
            match_index = future_rows[future_rows['High'] == defence].index
            if not match_index.empty:
                match_index = match_index[0]
                trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) - 1

    return trade_data

In [35]:
# Apply the function to the DataFrame
trade_data_defence= process_trade_data_defence(trade_data)
cal_data_defence= process_trade_data_defence(trade_data)
ideal_data_defence= process_trade_data_defence(trade_data)
trade_data_defence.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/trade_defence_target.csv", index=False)
ideal_data_defence.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/ideal_defence_target.csv", index=False)
cal_data_defence.to_csv("/content/drive/MyDrive/Real_Trader_EDA_R1/Target_Optimization_data/cal_defence_target.csv", index=False)

<ipython-input-34-228646b7ee8c>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3239.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) - 1
<ipython-input-34-228646b7ee8c>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3239.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) - 1
<ipython-input-34-228646b7ee8c>:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3239.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_data.at[i, 'defence_target'] = ((entry + defence) / 2) - 1
